In [ ]:
!pip install datasets
!pip install transformers==4.3.3
!pip install sentencepiece
#only needed for AraBERTv1 and v2
!pip install farasapy
!pip install pyarabic
!pip install fuzzysearch

     |████████████████████████████████| 306 kB 4.3 MB/s 
     |████████████████████████████████| 133 kB 90.7 MB/s 
     |████████████████████████████████| 243 kB 95.4 MB/s 
     |████████████████████████████████| 1.1 MB 79.3 MB/s 
     |████████████████████████████████| 67 kB 7.4 MB/s 
     |████████████████████████████████| 160 kB 57.1 MB/s 
     |████████████████████████████████| 271 kB 84.5 MB/s 
     |████████████████████████████████| 144 kB 77.1 MB/s 
     |████████████████████████████████| 1.9 MB 4.2 MB/s 
     |████████████████████████████████| 895 kB 73.6 MB/s 
     |████████████████████████████████| 3.3 MB 86.4 MB/s 
     |████████████████████████████████| 1.2 MB 4.2 MB/s 
     |████████████████████████████████| 126 kB 4.0 MB/s 
     |████████████████████████████████| 112 kB 4.2 MB/s 
  Created wheel for fuzzysearch: filename=fuzzysearch-0.7.3-cp37-cp37m-linux_x86_64.whl size=280742 sha256=79bcf9f34c32a6f6188c244f822b2fa260f62b03540d550b910dad8b71e88b71
  Stored in directory: 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt 
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import math 
import random
import torch
import torchvision
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import tensorflow as tf
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from scipy import stats
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from transformers.models.bert.modeling_bert import BertModel,BertForMaskedLM
matplotlib.rcParams['figure.facecolor'] = '#ffffff'
%matplotlib inline

In [ ]:
data=open("art.txt")

In [ ]:
data

<_io.TextIOWrapper name='art.txt' mode='r' encoding='UTF-8'>

In [ ]:
# open text file and read in data as `text`
with open('art.txt', 'r',encoding="utf-8") as f:
    text = f.read()

In [ ]:
 text[:100]

'يابير طيك طي واعماقك اعماق\n\nويعيش بك داب وحمامه وعصفور\n\nفاضت عيونك ماك وابتلت احداق\n\nوتنهدت منك محال'

In [ ]:
dataset = MNIST(root='data/',download =True,transform = ToTensor())

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [ ]:
#Splitting dataset to train and validaion

train_size = len(dataset) 

train_set = random_split(dataset,[train_size])

In [ ]:
torch.cuda.is_available()

True

In [ ]:
batch_size = 128
train_loader = DataLoader(train_set,batch_size=128,shuffle=True,num_workers=2,pin_memory =True)

In [ ]:
train_loader = DeviceDataLoader(train_loader,device)
val_loader = DeviceDataLoader(val_loader,device)

NameError: ignored

In [ ]:
class Mnistmodel(nn.Module):
  def __init__(self,input_size,hidden_size,out_size):
    super().__init__()
    ##hidden layer 
    self.linear1 = nn.Linear(input_size,hidden_size)
    ## Output layer
    self.linear2 = nn.Linear(hidden_size,out_size)
  def forward(self,xb):
    xb = xb.view(xb.size(0),-1)
    # Output of hidden layer
    out = self.linear1(xb)
    #Applying Relu activation on output of hidden layer
    rel = F.relu(out)
    # Final output
    out  = self.linear2(rel)
    return out
  def training_step(self,batch):
    """ Returns loss for a training data"""
    images,labels = batch
    out = self(images)
    loss = F.cross_entropy(out,labels)
    return loss
  def validation_step(self,batch):
    """Finding loss and accuracy for a batch of validation data"""
    images,labels = batch
    out = self(images)
    loss = F.cross_entropy(out,labels)
    acc = accuracy(out,labels)
    return {'val_loss':loss,'val_acc':acc}
  def validation_epoch_end(self,outputs):
    batch_losses = [x['val_loss'] for x in outputs]
    epoch_loss = torch.stack(batch_losses).mean()
    batch_accuracy = [x['val_acc'] for x in outputs]
    epoch_accuracy = torch.stack(batch_accuracy).mean()
    return {'val_loss':epoch_loss,'val_acc':epoch_accuracy}
  def epoch_end(self,epoch,result):
    print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))


In [ ]:
def fit(epochs,lr,model,train,opt_func = torch.optim.SGD):
  history = []
  optimizer = opt_func(model.parameters(),lr)
  for epoch in range(epochs):
    #training phase
    for batch in train_loader:
      loss = model.training_step(batch)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

  return history



In [ ]:
num_classes =10
model = Mnistmodel(input_size =784 ,hidden_size=32,out_size= num_classes)

In [ ]:
history = [eval(model)]
print(history)

TypeError: ignored

In [ ]:
model_name="aubmindlab/bert-base-arabertv02"

In [ ]:
model_name+=fit(5, 0.5, model, train_set)

TypeError: ignored